In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords


In [2]:
#Get the spam data collection 
dfSpamCollection = pd.read_csv('S:\SandeepG\Official\DataScience\SimpliLearn\Data Science with Python\Lesson 09 NLP\9.7 Practice Project\SpamCollection', sep='\t',
                              names=['response','message'])

In [3]:
dfSpamCollection.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
dfSpamCollection.describe()

,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [5]:
#view response 
dfSpamCollection['response']

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: response, Length: 5572, dtype: object

In [6]:
dfSpamCollection.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [7]:
#Verify length of the messages and also add it as a new column 
dfSpamCollection['length'] = dfSpamCollection['message'].apply(len)


In [8]:
dfSpamCollection.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [9]:
#define a function to get rid of stopwords present in the messages
def removeStopWords(message):
    puncLessMessage = [char for char in message if char not in string.punctuation]
    puncLessMessage = ''.join(puncLessMessage)
    return [word for word in puncLessMessage.split() if word.lower() not in stopwords.words('english')]

In [10]:
dfSpamCollection['message'].head(5).apply(removeStopWords)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [11]:
#start text processing with vectorizer 
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
#use bag of words by applying the function and fit the data into it
bagOfWordsTransformer = CountVectorizer(analyzer=removeStopWords).fit(dfSpamCollection['message'])

In [13]:
#print length of bag of words stored in the vocabulary_ attribute
print(len(bagOfWordsTransformer.vocabulary_))

11425


In [14]:
message_bagOfWords = bagOfWordsTransformer.transform(dfSpamCollection['message'])

In [15]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(message_bagOfWords)

In [16]:
#print shape of the tfidf 
message_tfidf = tfidf_transformer.transform(message_bagOfWords)
print(message_tfidf.shape)

(5572, 11425)


In [17]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spamDetectModel = MultinomialNB().fit(message_tfidf, dfSpamCollection['response'])

In [18]:
#check model for the predicted and expected value say for message#2 and message#5

# predict for message 2
message = dfSpamCollection['message'][1]
bagOfWordsForMessage = bagOfWordsTransformer.transform([message])
tfidf = tfidf_transformer.transform(bagOfWordsForMessage)

print('Predicted for message#2 :', spamDetectModel.predict(tfidf)[0])
print('Expected for message#2 :', dfSpamCollection.response[0])

# predict for message 5
message = dfSpamCollection['message'][4]
bagOfWordsForMessage = bagOfWordsTransformer.transform([message])
tfidf = tfidf_transformer.transform(bagOfWordsForMessage)

print('Predicted for message#5 :', spamDetectModel.predict(tfidf)[0])
print('Expected for message#5 :', dfSpamCollection.response[0])


Predicted for message#2 : ham
Expected for message#2 : ham
Predicted for message#5 : ham
Expected for message#5 : ham
